# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

#### Prerequisites

We will be using a basic agent with a search tool here.

In [2]:
%pip install -U --quiet  langgraph langchain-fireworks
%pip install -U --quiet tavily-python
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


# Optional: Configure tracing to visualize and debug the agent
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("GOOGLE_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflection"

# _set_if_undefined("FIREWORKS_API_KEY")

## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:


In [4]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks
from langchain_google_genai import ChatGoogleGenerativeAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
# llm = ChatFireworks(
#     model="accounts/fireworks/models/mixtral-8x7b-instruct",
#     model_kwargs={"max_tokens": 32768},
# )

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
generate = prompt | llm

/workspaces/langgraph/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

## The Enduring Relevance of "The Little Prince" in Modern Childhood

Antoine de Saint-Exupéry's "The Little Prince" is a timeless classic that continues to resonate with readers of all ages, particularly children. While the world has changed drastically since its publication in 1943, the themes explored in this poignant tale remain profoundly relevant to modern childhood. The story's enduring appeal lies in its ability to speak to the universal experiences of childhood, exploring themes of imagination, friendship, loss, and the search for meaning in a complex world. 

The Little Prince's journey across the cosmos allows children to embrace the power of imagination and wonder. His encounters with diverse planets and their inhabitants offer a unique perspective on the world, encouraging young readers to question the ordinary and explore the extraordinary. This emphasis on imagination is crucial in a world increasingly dominated by technology and instant gratification, where the power of

### Reflect

In [6]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [7]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

This is a good start to an essay on the relevance of "The Little Prince" in modern childhood. You have identified several key themes and provided some insightful observations. However, there are areas where you can expand and strengthen your argument. 

Here are some recommendations:

**1. Expand on Specific Examples:**

* **Imagination:** Instead of just saying it's important, give specific examples of how the Little Prince's journey encourages imagination. For instance, discuss how the planets and their inhabitants (like the king, the conceited man, or the lamplighter) stimulate the reader's imagination. How does this relate to modern children's need for creativity in a world dominated by technology?
* **Friendship:**  Elaborate on the relationship between the Little Prince and the aviator. How does it illustrate the value of genuine connection and how does this relate to the challenges of building friendships in a digital age? 
* **Loss and Grief:**  Dive deeper into the Prince's re

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [8]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

## The Little Prince: A Timeless Guide for Modern Childhood

Antoine de Saint-Exupéry's "The Little Prince" is more than just a charming children's story; it's a timeless masterpiece that speaks directly to the heart of childhood, even in our increasingly complex and digital world. The story's enduring appeal lies in its exploration of universal themes like imagination, friendship, loss, and the search for meaning – themes that remain profoundly relevant to children navigating the challenges of modern life. 

The Little Prince's journey across the cosmos is a powerful testament to the importance of imagination. Each planet he visits, from the king's tiny kingdom to the conceited man's vanity, and the lamplighter's endless task, sparks the reader's imagination, encouraging them to see the world with fresh eyes and question the ordinary. In a world saturated with technology, where instant gratification and virtual experiences often dominate, "The Little Prince" reminds children of the po

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [9]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [10]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content="## The Enduring Relevance of The Little Prince: A Timeless Tale for Modern Times\n\nAntoine de Saint-Exupéry's *The Little Prince* is a beloved children's book that transcends age and cultural boundaries. Published in 1943, the story of a young prince who travels from asteroid to asteroid, seeking meaning and connection, resonates deeply with readers of all ages. While the book may seem simplistic on the surface, its themes of imagination, love, and the importance of seeing the world with a childlike perspective remain profoundly relevant in the complexities of modern life.\n\nOne of the most enduring messages of *The Little Prince* is the importance of cherishing the power of imagination. The prince's journey is fueled by his unbridled curiosity and his ability to see beauty in the ordinary. He finds wonder in a simple rose, a volcanic mountain, and a fox's friendship. In a world increasingly dominated by technology and digital distractions, this remind

In [13]:
event["generate"]

AIMessage(content='## The Little Prince: A Timeless Tale for a Disconnected World\n\nAntoine de Saint-Exupéry\'s *The Little Prince*, a seemingly simple children\'s book, continues to offer profound insights into the human condition, particularly in the context of modern life.  This essay will explore how the book\'s themes of imagination, connection, and the pursuit of meaning resonate deeply with the anxieties and challenges of our contemporary world, reminding us of the enduring importance of cherishing the simple joys and seeking genuine connection in a world often consumed by complexity and superficiality.\n\nOne of the most poignant aspects of *The Little Prince* is its celebration of imagination and wonder. The prince\'s journey is fueled by his insatiable curiosity and his ability to see beauty in the ordinary. He finds wonder in a simple rose, its delicate petals and intoxicating fragrance a testament to the power of nature. He marvels at the volcanic mountains, their fiery er

In [15]:
print(event["generate"].content)

## The Little Prince: A Timeless Tale for a Disconnected World

Antoine de Saint-Exupéry's *The Little Prince*, a seemingly simple children's book, continues to offer profound insights into the human condition, particularly in the context of modern life.  This essay will explore how the book's themes of imagination, connection, and the pursuit of meaning resonate deeply with the anxieties and challenges of our contemporary world, reminding us of the enduring importance of cherishing the simple joys and seeking genuine connection in a world often consumed by complexity and superficiality.

One of the most poignant aspects of *The Little Prince* is its celebration of imagination and wonder. The prince's journey is fueled by his insatiable curiosity and his ability to see beauty in the ordinary. He finds wonder in a simple rose, its delicate petals and intoxicating fragrance a testament to the power of nature. He marvels at the volcanic mountains, their fiery eruptions a symbol of life's 

## Conclusion

Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherently cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc.). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.
